# Objective

- Create a Sensitive Personally Identifiable Information (SPII) detector
- The model will receive a customer-agent conversation and will return a masked conversation with the sensitive information replaced by a generic token.
- Sensitve information includes:
    - Names
    - Phone Numbers
    - Email Addresses
    - Social Security Numbers
    - Addresses


### Example


#### Input

```
**الموظف:** مرحبًا! شكرًا لتواصلك مع خدمة العملاء. كيف يمكنني مساعدتك اليوم؟
**علي:** مرحبًا، أودّ تحديث بياناتي في النظام.
**الموظف:** بالتأكيد، يمكننا مساعدتك في ذلك. سأحتاج إلى بعض المعلومات منك. هل يمكنك تزويدي باسمك الكامل؟
**علي:** نعم، اسمي علي.
**الموظف:** شكرًا لك، علي. ما هو عنوان إقامتك؟
**علي:** أنا أسكن في الحمراء.
**الموظف:** رائع. هل يمكنك تزويدي برقم هاتفك؟
**علي:** نعم، رقمي هو 07772881.
**الموظف:** شكرًا، وأخيرًا، سأحتاج إلى رقم الضمان الاجتماعي الخاص بك للتحقق من حسابك.
**علي:** بالتأكيد، رقمي هو 2240-120210.
**الموظف:** شكرًا لك، تم تحديث بياناتك بنجاح. هل هناك أي شيء آخر يمكنني مساعدتك به؟
```

#### Output

```
**الموظف:** مرحبًا! شكرًا لتواصلك مع خدمة العملاء. كيف يمكنني مساعدتك اليوم؟
**علي:** مرحبًا، أودّ تحديث بياناتي في النظام.
**الموظف:** بالتأكيد، يمكننا مساعدتك في ذلك. سأحتاج إلى بعض المعلومات منك. هل يمكنك تزويدي باسمك الكامل؟
**علي:** نعم، اسمي *****.
**الموظف:** شكرًا لك، *****. ما هو عنوان إقامتك؟
**علي:** أنا أسكن في ****.
**الموظف:** رائع. هل يمكنك تزويدي برقم هاتفك؟
**علي:** نعم، رقمي هو *********.
**الموظف:** شكرًا، وأخيرًا، سأحتاج إلى رقم الضمان الاجتماعي الخاص بك للتحقق من حسابك.
**علي:** بالتأكيد، رقمي هو **********.
**الموظف:** شكرًا لك، تم تحديث بياناتك بنجاح. هل هناك أي شيء آخر يمكنني مساعدتك به؟
```

# Using Pretrained Models

You can find a list of all available models on <https://huggingface.co/models>.

In this notebook we are going to use <https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct>.

## Model Initialization

In [ ]:
from transformers import AutoModelForCausalLM

BASE_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    cache_dir="./models",
    torch_dtype="auto",
    device_map="auto",
)

## Tokenization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

In [ ]:
SYSTEM_PROMPT = """\
You are an expert in detecting Sensitive Personally Identifiable Information (SPII).
Sensitive data include: names, email addresses, addresses, phone numbers, and social security numbers.
Your objective is to analyze a user message, and return a masked version with all SPII data replaced with a '*'
"""

EXAMPLE = """\
**الموظف:** مرحبًا! شكرًا لتواصلك مع خدمة العملاء. كيف يمكنني مساعدتك اليوم؟
**علي:** مرحبًا، أودّ تحديث بياناتي في النظام.
**الموظف:** بالتأكيد، يمكننا مساعدتك في ذلك. سأحتاج إلى بعض المعلومات منك. هل يمكنك تزويدي باسمك الكامل؟
**علي:** نعم، اسمي علي.
**الموظف:** شكرًا لك، علي. ما هو عنوان إقامتك؟
**علي:** أنا أسكن في الحمراء.
**الموظف:** رائع. هل يمكنك تزويدي برقم هاتفك؟
**علي:** نعم، رقمي هو 07772881.
**الموظف:** شكرًا، وأخيرًا، سأحتاج إلى رقم الضمان الاجتماعي الخاص بك للتحقق من حسابك.
**علي:** بالتأكيد، رقمي هو 2240-120210.
**الموظف:** شكرًا لك، تم تحديث بياناتك بنجاح. هل هناك أي شيء آخر يمكنني مساعدتك به؟
"""

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": EXAMPLE},
]

In [ ]:
formatted_msg = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
tokens = tokenizer(formatted_msg, return_tensors="pt").to(model.device)

In [ ]:
print(tokens)

In [ ]:
print(formatted_msg)

## Pipeline

In [ ]:
from transformers import TextStreamer

def pipeline(model, tokenizer, messages):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024,
        do_sample=False,
        temperature=None,
        top_k=None,
        top_p=None,
        streamer=streamer,
    )
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [ ]:
response = pipeline(model, tokenizer, messages)

In [ ]:
print(response)

## Comparison

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


def run_openai(system_prompt: str, message: str):
    prompt = ChatPromptTemplate.from_messages(
        messages=[("system", system_prompt), ("human", "{message}")]
    )
    chain = prompt | ChatOpenAI(model_name="gpt-4o-mini")
    return chain.invoke(input={"message": message})

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
response = run_openai(SYSTEM_PROMPT, EXAMPLE)
print(response.content)

# Preparing Training Data

## Scenario Generation

In [ ]:
import json
import tqdm

In [ ]:
def scenario_generation():
    prompt = ChatPromptTemplate.from_messages(
        messages=[
            (
                "human",
                (
                    "Your job is generate a fake customer-support/agent conversation in Arabic language."
                    "The interaction must contain SPII information of the customer."
                    f"Here is an example: {EXAMPLE}."
                ),
            ),
        ]
    )
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0.6)
    chain = prompt | model
    return chain.invoke(input={})


In [ ]:
scenarios = []
NB_SCENARIOS = 1
for i in tqdm.trange(NB_SCENARIOS):
    r = scenario_generation()
    scenarios.append(r.content)

In [ ]:
print(scenarios[0])

In [ ]:
data = [{"input": s} for s in scenarios]
data_json = json.dumps(scenarios, ensure_ascii=False)
with open("outputs/scenarios.json", "w") as f:
    f.write(data_json)

## Data Labeling

In [ ]:
response = run_openai(SYSTEM_PROMPT, scenarios[0])
print(response.content)

In [ ]:
labels = []
for scenario in tqdm.tqdm(scenarios):
    response = run_openai(system_prompt=SYSTEM_PROMPT, message=scenario)
    labels.append({
      "instruction": SYSTEM_PROMPT,
      "input": scenario,
      "output": response.content
      })

In [ ]:
with open("outputs/labels.json", "w") as f:
    json.dump(labels, f, ensure_ascii=False)

# Fine Tuning

## Installing LLaMA-Factory

```bash
git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory
cd LLaMA-Factory
uv sync --python 3.12 --extra torch --extra metrics --prerelease=allow
```

## Dataset Definition

Add the following to `dataset_info.json`:
```json
{
  ...
  "customer_support": {
    "file_name": "/path/to/labels.json",
    "columns": {
      "prompt": "instruction",
      "query": "input",
      "response": "output"
    }
  },
  ...
}
````

## Training Job Configuration

You can download the template from: <https://github.com/hiyouga/LLaMA-Factory/blob/main/examples/train_lora/llama3_lora_sft.yaml>

```yaml
### model
model_name_or_path: Qwen/Qwen2.5-1.5B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 8
lora_target: all

### dataset
dataset: customer_support
template: qwen
cutoff_len: 2048
overwrite_cache: false
preprocessing_num_workers: 16

### output
output_dir: lora-adapter/
logging_steps: 10
save_steps: 500
# plot_loss: true
# overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000

### eval
val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 500

report_to: none

# 
# export HF_TOKEN="ADD_TOKEN_HERE"
push_to_hub: true
export_hub_model_id: alimasri/Qwen2.5-1.5B-Instruct-ar-spii
hub_strategy: checkpoint
hub_private_repo: false
```

## Start Finetuning!

```bash
uv run --prerelease=allow llamafactory-cli train /path/to/config.yaml
```

## Using the Adapter

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    cache_dir="./models",
    torch_dtype="auto",
    device_map="auto",
)

In [ ]:
from peft import PeftModel

adapter_id = 'alimasri/Qwen2.5-1.5B-Instruct-ar-spii'

finetuned_model = PeftModel.from_pretrained(base_model, adapter_id)
# OR use the `load_adapter` method
# model.load_adapter(adapter_id)


In [ ]:
response = pipeline(finetuned_model, tokenizer, messages)

In [ ]:
print(response)